# Autorzy
- Mateusz Łopaciński
- Mateusz Mazur

# Wczytanie danych

In [2]:
from scipy.io import loadmat

data = loadmat('./data/leukemia.mat')
X = data['X']
y = data['Y'].ravel()

# Usuwanie cech charakteryzujących się niską wariancją

## Importujemy potrzebne biblioteki

In [3]:
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

## 

Dzielimy zbiór na treningowy i testowy.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Przyjmujemy próg dla wariancji, poniżej którego cechy będą usuwane.

In [5]:
threshold = 3

Obliczamy wariancję i do dalszych predykcji używamy tylko te cechy, spełniające warunek.

In [6]:
selector = VarianceThreshold(threshold)
X_transformed = selector.fit_transform(X)

X_train_high_variance, X_test_high_variance, y_train, y_test = train_test_split(X_transformed, y, test_size=0.3, random_state=42)

In [9]:
X_transformed.shape

(72, 14)

Porównanie skuteczności przed i po zmianach.

In [10]:
# Original dataset
clf_original = RandomForestClassifier(random_state=42)
clf_original.fit(X_train, y_train.ravel())
y_pred_original = clf_original.predict(X_test)
accuracy_original = accuracy_score(y_test, y_pred_original)

# Dataset after removing low variance features
clf_high_variance = RandomForestClassifier(random_state=42)
clf_high_variance.fit(X_train_high_variance, y_train.ravel())
y_pred_high_variance = clf_high_variance.predict(X_test_high_variance)
accuracy_high_variance = accuracy_score(y_test, y_pred_high_variance)

In [11]:
print("Skuteczność na oryginalnym zbiorze: {:.2f}".format(accuracy_original * 100))
print("Skuteczność po usunięciu cech o niskiej wariancji: {:.2f}".format(accuracy_high_variance * 100))

Skuteczność na oryginalnym zbiorze: 100.00
Skuteczność po usunięciu cech o niskiej wariancji: 95.45


Jak widzimy przy użyciu tylko 14 cech, zamiast 7070, nadal uzyskujemy wyniki na dobrym poziomie.
# TODO

# Algorytm rekursywnej eliminnacji cech


## Importujemy potrzebne biblioteki

In [95]:
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

## Obliczamy próg `m`

In [96]:
n_instances = X_train_high_variance.shape[0]
m = n_instances // 3

Używając dwóch klasyfikatorów, Regresji Logistycznej oraz Lasów Losowych, przy pomocy RFE wyznaczammy najważniejsze cechy.

In [97]:
estimator_lr = LogisticRegression(solver='lbfgs', max_iter=1000)
rfe_lr = RFE(estimator=estimator_lr, n_features_to_select=m)
X_train_selected_lr = rfe_lr.fit_transform(X_train_high_variance, y_train.ravel())
X_test_selected_lr = rfe_lr.transform(X_test_high_variance)

In [98]:
estimator_rf = RandomForestClassifier(random_state=42)
rfe_rf = RFE(estimator=estimator_rf, n_features_to_select=m)
X_train_selected_rf = rfe_rf.fit_transform(X_train_high_variance, y_train.ravel())
X_test_selected_rf = rfe_rf.transform(X_test_high_variance)

## Sprawdzenie dokładności oraz AUC dla powyższych klasyfikatorów z wykorzystaniem 4-krotnej walidacji krzyżowej oraz przetransformowanych danych

Importujemy potrzebne biblioteki

In [99]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score

4-krotna walidacja krzyżowa z wyliczaniem dokładności i AUC dla dwóch klasyfikatorów

In [100]:
n_splits = 4
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
rf = RandomForestClassifier(random_state=42)

cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

mean_accuracy_lr = np.mean(cross_val_score(lr, X_transformed, y.ravel(), cv=cv, scoring='accuracy'))
mean_accuracy_rf = np.mean(cross_val_score(rf, X_transformed, y.ravel(), cv=cv, scoring='accuracy'))

mean_auc_lr = np.mean(cross_val_score(lr, X_transformed, y.ravel(), cv=cv, scoring='roc_auc'))
mean_auc_rf = np.mean(cross_val_score(rf, X_transformed, y.ravel(), cv=cv, scoring='roc_auc'))

In [101]:
print("Średnia dokładność dla regresji logistycznej: {:.2f}".format(mean_accuracy_lr * 100))
print("Średnia dokładność dla lasów losowych: {:.2f}".format(mean_accuracy_rf * 100))

print("Średnia metryka AUC dla regresji logistycznej: {:.2f}".format(mean_auc_lr * 100))
print("Średnia metryka AUC dla lasów losowych: {:.2f}".format(mean_auc_rf * 100))

Średnia dokładność dla regresji logistycznej: 93.06
Średnia dokładność dla lasów losowych: 93.06
Średnia metryka AUC dla regresji logistycznej: 98.29
Średnia metryka AUC dla lasów losowych: 99.31


Dla lasów losowych uzyskaliśmy wyższy wynik AUC, niż dla regresji logistycznej. Dokładność obu klasyfikatorów jest bardzo zbliżona

# Wbudowane metody selekcji cech

In [102]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [103]:
estimator_l1 = LogisticRegression(solver='saga', penalty='l1', C=0.1, max_iter=1000) # regresja logistyczna z regularyzacją L1
selector_l1 = SelectFromModel(estimator_l1)  # selektor używający metody regularyzacji L1
X_train_l1 = selector_l1.fit_transform(X_train_high_variance, y_train.ravel())
X_test_l1 = selector_l1.transform(X_test_high_variance)

In [104]:
estimator_rf = RandomForestClassifier(random_state=42) # estymator lasów losowych
selector_rf = SelectFromModel(estimator_rf)  # selektor używający metody opartej na ważności cech dla lasów losowych
X_train_rf = selector_rf.fit_transform(X_train_high_variance, y_train.ravel())
X_test_rf = selector_rf.transform(X_test_high_variance)

In [105]:
estimator_rf = RandomForestClassifier(random_state=42) # estymator lasów losowych
selector_rf = SelectFromModel(estimator_rf)  # selektor używający metody opartej na ważności cech dla lasów losowych
X_train_rf = selector_rf.fit_transform(X_train_high_variance, y_train.ravel())
X_test_rf = selector_rf.transform(X_test_high_variance)

In [106]:
# Oryginalna regresja logistyczna
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
lr.fit(X_train_high_variance, y_train.ravel())
y_pred_lr = lr.predict(X_test_high_variance)
accuracy_lr = accuracy_score(y_test, y_pred_lr)

# Oryginalne lasy losowe
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_high_variance, y_train.ravel())
y_pred_rf = rf.predict(X_test_high_variance)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

# Regularyzacja L1 dla regresji logistycznej
lr_l1 = LogisticRegression(solver='lbfgs', max_iter=1000)
lr_l1.fit(X_train_l1, y_train.ravel())
y_pred_l1 = lr_l1.predict(X_test_l1)
accuracy_l1 = accuracy_score(y_test, y_pred_l1)

# Metoda oparta na ważności cech dla lasów losowych
rf_selected = RandomForestClassifier(random_state=42)
rf_selected.fit(X_train_rf, y_train.ravel())
y_pred_rf_selected = rf_selected.predict(X_test_rf)
accuracy_rf_selected = accuracy_score(y_test, y_pred_rf_selected)

In [109]:
print("Dokładność regresji logistycznej: {:.2f}".format(accuracy_lr * 100))
print("Dokładność lasów losowych: {:.2f}".format(accuracy_rf * 100))
print("Dokładność regresji logistycznej z regularyzacją L1: {:.2f}".format(accuracy_l1 * 100))
print("Dokładność lasów losowych z metodą ważności cech: {:.2f}".format(accuracy_rf_selected * 100))

Dokładność regresji logistycznej: 95.45
Dokładność lasów losowych: 95.45
Dokładność regresji logistycznej z regularyzacją L1: 90.91
Dokładność lasów losowych z metodą ważności cech: 86.36


In [110]:
import scipy.io
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# 1. Wczytaj zbiór danych
data = scipy.io.loadmat('data/leukemia.mat')
X = data['X']
Y = np.ravel(data['Y'])

# 2. Usuń cechy o niskiej wariancji
var_thresh = VarianceThreshold(threshold=1.0)
X_high_variance = var_thresh.fit_transform(X)

# 3. Wybierz `m` najlepszych cech
m = X_high_variance.shape[0] // 3
estimators = [LogisticRegression(solver='liblinear'), RandomForestClassifier()]

rfe_models = {}
for est in estimators:
    rfe = RFE(estimator=est, n_features_to_select=m)
    X_selected = rfe.fit_transform(X_high_variance, Y)
    rfe_models[type(est).__name__] = {'estimator': est, 'X_selected': X_selected}

# 4. Zbadaj wpływ metryk na dokładność klasyfikacji
nr_splits = 6
scores = {}
for est_name, model_data in rfe_models.items():
    est = model_data['estimator']
    X_sel = model_data['X_selected']
    acc_scores = cross_val_score(est, X_sel, Y, cv=nr_splits, scoring='accuracy')
    auc_scores = cross_val_score(est, X_sel, Y, cv=nr_splits, scoring='roc_auc')
    scores[est_name] = {'accuracy': acc_scores, 'auc': auc_scores}
    
# 5. Porównaj skuteczność z wbudowanymi metodami selekcji cech
# (a) Regularyzacja L1 dla regresji logistycznej
est_l1 = LogisticRegression(penalty='l1', solver='liblinear')
sfm_l1 = SelectFromModel(est_l1)
X_sfm_l1 = sfm_l1.fit_transform(X, Y)

# (b) Ważność cech dla lasów losowych
est_rf = RandomForestClassifier()
sfm_rf = SelectFromModel(est_rf)
X_sfm_rf = sfm_rf.fit_transform(X, Y)

sfm_models = {'LogisticRegression_L1': {'estimator': est_l1, 'X_selected': X_sfm_l1},
              'RandomForest_FI': {'estimator': est_rf, 'X_selected': X_sfm_rf}}

sfm_scores = {}
for est_name, model_data in sfm_models.items():
    est = model_data['estimator']
    X_sel = model_data['X_selected']
    acc_scores = cross_val_score(est, X_sel, Y, cv=nr_splits, scoring='accuracy')
    auc_scores = cross_val_score(est, X_sel, Y, cv=nr_splits, scoring='roc_auc')
    sfm_scores[est_name] = {'accuracy': acc_scores, 'auc': auc_scores}

# Porównanie skuteczności - dokładność
print("Scores for RFE models:")
for model_name, score_data in scores.items():
    print(f"{model_name}: accuracy={np.mean(score_data['accuracy']):.2f}, auc={np.mean(score_data['auc']):.2f}")

print("\nScores for SFM models:")
for model_name, score_data in sfm_scores.items():
    print(f"{model_name}: accuracy={np.mean(score_data['accuracy']):.2f}, auc={np.mean(score_data['auc']):.2f}")